In [1]:
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go
from statsmodels.tsa.statespace.sarimax import SARIMAX
from glob import glob
import re
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/javier/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning:

pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy



In [3]:
#EXTRAER LOS NOMBRES DE TODOS LOS ARCHIVOS PARA SU POSTERIOR LECTURA Y ORDENARLOS EN ORDEN CRONOLÓGICO
archivos = glob('/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/*Semana*.txt')

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):

    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

archivos.sort(key=natural_keys)

In [4]:
#CREAMOS LOS CSVs VACÍOS PARA ALMACENAR LOS DFs DE CADENA PARA SARIMAX
data_prueba = pd.DataFrame(columns=['Target'])
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/atres.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/sexta.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/neox.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/nova.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/mega.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/atreseries.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/telecinco.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/cuatro.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/divinity.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/energy.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/fdf.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/bemad.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/dmax.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/dkiss.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/paramount.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/ten.csv')
data_prueba.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/trece.csv')

data_ml = pd.DataFrame(columns=['Full_date','Time','Date', 'Day_name', 'Month_name','Year','Time_frame'
                                ,'Channel','Sales_house','Title/Descrip', 'Genre','Public','Production_comp'
                                ,'Age_rating','Adults_avg_audience'])
data_ml.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/ML_ALGOR/df_ml.csv')

In [ ]:
#FUNCIÓN PARA CREAR LA COLUMNA DE FRANJAS HORARIAS A PARTIR DE VALOR DE FECHA.
def time_frame (value):
    
    if ((value.time() >= datetime.time(7,0)) & (value < datetime.time(14,0))):
        return 'Morning'
    if ((value.time() >= datetime.time(14,0)) & (value < datetime.time(16,0))):
        return 'Lunch'
    if ((value.time() >= datetime.time(16,0)) & (value < datetime.time(20,30))):
        return 'After'
    if ((value.time() >= datetime.time(20,30)) & (value <= datetime.time(23,59))):
        return 'PT'
    if ((value.time() >= datetime.time(0,0)) & (value < datetime.time(0,30))):
        return 'PT'
    if ((value.time() >= datetime.time(0,30)) & (value < datetime.time(1,30))):
        return 'Late_PT'
    if ((value.time() >= datetime.time(1,30)) & (value < datetime.time(7,0))):
        return 'Night'
    

#FUNCIÓN PARA CREAR EL GRUPO COMERCIAL DE CADA CADENA
def com_group(cadena):
    atres = ['A3','LA SEXTA','NEOX','NOVA','MEGA','ATRESERIES']
    medset = ['T5','CUATRO','DIVINITY','ENERGY','FDF','BEMADtv']
    pulsa = ['DMAX','DKISS','PARAMOUNT NETWORK','TEN','TRECE']
    
    if cadena in atres:
        return 'ATRESMEDIA'
    if cadena in medset:
        return 'MEDIASET'
    if cadena in pulsa:
        return 'PULSA'
    


In [ ]:
%%timeit
counter = 0
for arch in archivos: 
    print(arch)
    #pd.set_option('display.max_columns', None)
    #pd.set_option('display.max_rows', None)
    df = pd.read_csv(arch,error_bad_lines=False,encoding = 'unicode_escape', 
                     header=3, sep='\t', skiprows=[4], skipinitialspace=True, 
                     usecols=['Título','Título/Descripción', 'Cadena', 'Fecha', 'Género','Público objetivo', 
                              'Productora','Calificación de Edad','Ind. 16+ (c/inv.)'],dtype ='str' )

    #ELIMINAMOS LAS CABECERAS DE LOS PROGRAMAS EN LA COLUMNA TÍTULO
    df.dropna(inplace = True)
    df = df[df.Título.str.startswith('<<')]
    df.reset_index(drop = True,inplace = True)

    #ELIMINAMOS LOS SIMBOLOS <> DEL DATA FRAME
    df1 = pd.DataFrame()
    for i in df.columns:
        df1 = pd.concat([df1,df.loc[:,i].str.translate({ord(x): None for x in ('><')})], axis = 1)

    #ESCOJO LAS CADENAS DE INTERÉS
    cadenas_v = ['A3','LA SEXTA','NEOX','NOVA','MEGA','ATRESERIES','T5','CUATRO','DIVINITY','ENERGY','FDF'
                 ,'BEMADtv','DMAX','DKISS','PARAMOUNT NETWORK','TEN','TRECE']
    df1 = df1[df1['Cadena'].isin(cadenas_v)]

    #DIVIDIMOS LA COLUMNA EN DOS PARA PODER OPERAR SOLO CON LA HORA 
    #(NO SE PUEDE PASAR A DATETIME YA QUE NO ESTÁ EN EL RANGO ADECUADO)
    df1[['Hora','Minuto']] = df1.Título.str.split(':', expand = True)
    #df1.head(5)

    #PASAMOS LA COLUMNA FECHA A DATETIME PARA PODER RESTAR DIAS CUANDO SEA NECESARIO
    df1['Fecha'] = pd.to_datetime(df1['Fecha'],format = "%d/%m/%Y", dayfirst=True)
    #df1.dtypes

    #SE CAMBIA EL TIPO DE DATO EN LAS COLUMNAS HORA MINUTO PARA PODER TRANSFORMAR A RANGO 00:00-23:00(STR-INT)
    df1 = df1.astype({'Hora':'int','Minuto':'int'})
    #df1.dtypes

    #SUMAMOS 1 DÍA A LAS HORAS SUPERIORES A 23
    ind = df1[df1['Hora']>23].index
    df1.loc[ind,'Fecha']= df1.loc[ind,'Fecha'] + timedelta(days=1)

    #CONVERTIMOS PARA TENER RANGO 00-23
    df1.Hora = df1.Hora.replace({24:0,25:1,26:2})

    #SE SUSTITUYE EL 24 POR 0 PARA TENER RANGO 00-23
    #df1.Hora = df1.Hora.replace(24,0)


    #CONVIERTO LA FECHA A STRING PARA PODER UNIRLO CON LA HORA
    df1['Fecha']=df1.Fecha.apply(lambda x:str(x.date()))
    df1[df1['Cadena']=='La1'].head(1413)

    #VOLVEMOS A CONVERTIR HORA Y MINUTO A STR PARA PODER UNIRLOS CON ':' COMO SEPARADOR
    df1 = df1.astype({'Hora':'str', 'Minuto':'str'})

    #UNIMOS HORA Y MINUTO PARA DESPUÉS UNIRLO A LA FECHA Y CONVERTIRLO EN TIMESTAMP
    df1 ['Tiempo'] = df1[['Hora', 'Minuto']].apply(lambda x: ':'.join(x), axis=1)
    df1 ['Fecha_comp'] = df1[['Fecha', 'Tiempo']].apply(lambda x: ' '.join(x), axis=1)
    df1.Fecha_comp = pd.to_datetime(df1.Fecha_comp)

    #ELIMINAMOS LAS COLUMNAS GENERADAS Y USADAS PARA LAS TRANSFORMACIONES
    df1 = df1.drop(['Tiempo','Hora', 'Minuto','Fecha','Título'], axis = 1)
    #,'Descripción'

    #CAMBIO COMA POR PUNTO PARA PODER CONVERTIR A ARRAY DE FLOAT
    #column = ['Ind. 4+ (inv)','Amas de Casa (c/inv.)', 'Ind. 4 - 12 (c/inv.)','Ind. 16+ (c/inv.)',
     #         'Ind. 4+ (inv).1','Amas de Casa (c/inv.).1','Ind. 4 - 12 (c/inv.).1', 'Ind. 16+ (c/inv.).1']
    column = 'Ind. 16+ (c/inv.)'
    df1['Ind. 16+ (c/inv.)']=df1['Ind. 16+ (c/inv.)'].str.replace(',','.')
   # for i in column:
    #    df1[i]=df1[i].str.replace(',','.')

    #CREO LA VARIABLE TARGET CON LAS COLUMNAS NECESARIAS
    df1['Target'] = df1['Ind. 16+ (c/inv.)'].apply(lambda x : float(x))
    #df1['Target']= df1[column].apply(lambda x:'/'.join(x),axis=1)
    #df1['Target']= df1.Target.str.split('/').apply(lambda x : [float(i) for i in x])
    #df1['Target']= df1.Target.str.split('/').apply(lambda x:np.array(x,dtype=np.float32))

    #ELIMINAMOS LAS VARIABLES UTILIZADAS EN TARGET
    df1 = df1.drop(column,axis = 1)

    #ELIMINAMOS TITULO/DESCRIPCION DE MOMENTO (!)
    #df1 = df1.drop('Título/Descripción',axis = 1)

    #ORDENO LAS COLUMNAS DE MI DATA FRAME (!) 
    df1 = df1[['Fecha_comp','Título/Descripción','Cadena', 'Género', 'Público objetivo', 'Productora',
           'Calificación de Edad', 'Target']]
    
    #ELIMINO LOS VALORES DUPLICADOS POR CADENA Y ME QUEDO CON LOS MÁXIMOS
    df1 = df1.groupby(['Fecha_comp','Cadena'])['Título/Descripción', 'Género', 'Público objetivo', 'Productora',
       'Calificación de Edad','Target'].agg({'Target':'max'}).reset_index(level=[0,1],col_level = 1)
    df1.columns = df1.columns.droplevel(level = 0)
    
    #AÑADIMOS LAS COLUMNAS NECESARIAS PARA REALIZAR LAS AGRUPACIONES DEL FRONTEND
    df1['Year'] = df1.Fecha_comp.dt.year
    df1['Date'] = df1.Fecha_comp.dt.date
    df1['Time'] = df1.Fecha_comp.dt.time
    df1['Day_name'] = df1.Fecha_comp.dt.day_name()
    df1['Month_name'] = df1.Fecha_comp.dt.month_name()

    df1['Sales_house'] = df1.Cadena.swifter.apply(lambda cadena: com_group(cadena))

    df1['Time_frame'] = df1.Fecha_comp.swifter.apply(lambda valor:time_frame(valor))
    

    #SELECCIONAMOS LAS COLUMNAS NECESARIAS PARA MODELO SARIMAX 
    pr = df1[['Fecha_comp','Cadena','Target']]
    
    
    #TRADUZCO LAS COLUMNAS PARA TRABAJAR EN INGLÉS
    df1 = df1.rename(columns={'Fecha_comp':'Full_date', 'Cadena':'Channel','Título/Descripción':'Title/Descrip',
                        'Género':'Genre','Público objetivo':'Public','Productora':'Production_comp'
                        ,'Calificación de Edad':'Age_rating','Target':'Adults_avg_audience'})
    
    #ORDENO LAS COLUMNAS COMO ME INTERESA
    df1 = df1[['Full_date', 'Time', 'Date', 'Day_name', 'Month_name', 'Year','Time_frame', 'Channel', 'Sales_house'
               ,'Title/Descrip', 'Genre','Public', 'Production_comp', 'Age_rating', 'Adults_avg_audience']]
    

    #SEPARAMOS DATAFRAMES EN CADENAS PARA DESARROLLAR LOS ALGORITMOS PERTINENTES PARA CADA UNO

    grupos = pr.groupby('Cadena')

    dfatrs = grupos.get_group('A3').groupby('Fecha_comp')['Target'].max().to_frame()
    dfsxt = grupos.get_group('LA SEXTA').groupby('Fecha_comp')['Target'].max().to_frame()
    dfnx = grupos.get_group('NEOX').groupby('Fecha_comp')['Target'].max().to_frame()
    dfnv = grupos.get_group('NOVA').groupby('Fecha_comp')['Target'].max().to_frame()
    dfmg = grupos.get_group('MEGA').groupby('Fecha_comp')['Target'].max().to_frame()
    dfatrser = grupos.get_group('ATRESERIES').groupby('Fecha_comp')['Target'].max().to_frame()
    dftcnc = grupos.get_group('T5').groupby('Fecha_comp')['Target'].max().to_frame()
    dfctr = grupos.get_group('CUATRO').groupby('Fecha_comp')['Target'].max().to_frame()
    dfdvnt = grupos.get_group('DIVINITY').groupby('Fecha_comp')['Target'].max().to_frame()
    dfnrg = grupos.get_group('ENERGY').groupby('Fecha_comp')['Target'].max().to_frame()
    dffdf = grupos.get_group('FDF').groupby('Fecha_comp')['Target'].max().to_frame()
    dfbmd = grupos.get_group('BEMADtv').groupby('Fecha_comp')['Target'].max().to_frame()
    dfdmx = grupos.get_group('DMAX').groupby('Fecha_comp')['Target'].max().to_frame()
    dfdks = grupos.get_group('DKISS').groupby('Fecha_comp')['Target'].max().to_frame()
    dfprmnt = grupos.get_group('PARAMOUNT NETWORK').groupby('Fecha_comp')['Target'].max().to_frame()
    dftn = grupos.get_group('TEN').groupby('Fecha_comp')['Target'].max().to_frame()
    dftrc = grupos.get_group('TRECE').groupby('Fecha_comp')['Target'].max().to_frame()
    print('proc_ok')
    
    #GUARDAMOS LOS DF DE CADENA EN ARCHIVOS CSV CREADOS PREVIAMENTE PARA CONCATENAR EL PROCESADO DE LOS DATOS
    dfatrs.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/atres.csv', mode = 'a', header = False)
    dfsxt.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/sexta.csv', mode = 'a', header = False)
    dfnx.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/neox.csv', mode = 'a', header = False)
    dfnv.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/nova.csv', mode = 'a', header = False)
    dfmg.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/mega.csv', mode = 'a', header = False)
    dfatrser.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/atreseries.csv', mode = 'a', header = False)
    dftcnc.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/telecinco.csv', mode = 'a', header = False)
    dfctr.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/cuatro.csv', mode = 'a', header = False)
    dfdvnt.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/divinity.csv', mode = 'a', header = False)
    dfnrg.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/energy.csv', mode = 'a', header = False)
    dffdf.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/fdf.csv', mode = 'a', header = False)
    dfbmd.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/bemad.csv', mode = 'a', header = False)
    dfdmx.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/dmax.csv', mode = 'a', header = False)
    dfdks.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/dkiss.csv', mode = 'a', header = False)
    dfprmnt.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/paramount.csv', mode = 'a', header = False)
    dftn.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/ten.csv', mode = 'a', header = False)
    dftrc.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/SARIMAX/trece.csv', mode = 'a', header = False)
    
    df1.to_csv('/home/javier/REPOS-GH/TFM-Javier_Casado/ML_ALGOR/df_ml.csv', mode = 'a' , header = False)
    counter += 1
    print('ok', counter)
    
    if counter == len(archivos):
        break
        

/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2018 - Semana 1 (07012018).txt


/home/javier/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:90: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



proc_ok
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2018 - Semana 2 (14012018).txt
proc_ok
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2018 - Semana 3 (21012018).txt
proc_ok
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2018 - Semana 4 (28012018).txt
proc_ok
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2018 - Semana 5 (04022018).txt
proc_ok
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2018 - Semana 6 (11022018).txt
proc_ok
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2018 - Semana 7 (18022018).txt
proc_ok
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2018 - Semana 8 (25022018).txt
proc_ok
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2018 - Semana 9 (04032018).txt
proc_ok
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2018 - Semana 10 (11032018).txt
proc_ok
ok
/home/javier/REPOS-GH/TFM-Javier_Casado/DATOS/2018 - Semana 11 (18032018).txt
